# Teoria de Numeros

Supongamos que se requiere trasmitir un mensaje codificado usando una
función $f : D \mapsto D$, donde $D = \{A, B, C , . . . , Z\}$. Dado que el alfabeto
contiene 26 letras, podemos usar el siguiente metodo:

\begin{align}
f(p) = (p + k) \operatorname{mod} 26
\end{align}

Donde $p$ es la poisición del carácter que vamos a reemplazar y $0 < k < 25$

In [152]:
import string


def encrypt(msg,private_key):
    enc_msg=''
    SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'
    code={v:k for k,v in enumerate(SYMBOLS)}
    reverse_code={k:v for k,v in enumerate(SYMBOLS)}
    for i in range(len(msg)):
        enc_msg+=reverse_code[(code[msg[i]]+private_key)%len(SYMBOLS)]
    return enc_msg

In [153]:
encrypt('CAESAR CIPHER DEMO',4)

'GEIWEVAGMTLIVAHIQS'

In [159]:
from secrets import token_bytes
from typing import Tuple

def random_key(length):
    tb = token_bytes(length)
    return int.from_bytes(tb, 'big')


In [160]:
def encrypt_one_time(msg):
    original_bytes = msg.encode()
    dummy = random_key(len(original_bytes))
    original_key = int.from_bytes(original_bytes, "big")
    encrypted = original_key ^ dummy # XOR
    return dummy, encrypted

In [161]:
(key1,key2)=encrypt_one_time('CAESAR CIPHER DEMO')

In [169]:
key2

2616896859878959746203082704444367061174740

In [170]:
def decrypt_one_time(key1, key2):
    decrypted = key1 ^ key2 # XOR
    temp= decrypted.to_bytes((decrypted.bit_length()+ 7) // 8, "big")
    return temp.decode()

In [171]:
decrypt_one_time(key1,key2)

'CAESAR CIPHER DEMO'

# Algoritmo RSA

El cifrado RSA se basa en la factorizacion de numeros enteros. A
diferencia del cifrado Cesar, RSA utiliza una llave publica $(e, n)$ y una privada $(d, n)$. 

Para codificar el mensaje m usamos la llave publica $e$ del receptor.

\begin{align}
c \equiv m^e  \operatorname{mod} n
\end{align}

Para decodificar el mensaje, usamos la llave privada $n$ del emisor.

\begin{align}
m \equiv c^d  \operatorname{mod} n
\end{align}

Los pasos para crear las llaves son:

* Crear de manera aleatoria dos enteros primos de gran tamano : $p$ y $q$.
* Multiplicar los enteros de manera de generar un numero compuesto $n$.
* Crear de manera aleatoria un entero $e$, que sea relativamente primo con $(p–1) \times (q–1)$. Es decir que el maximo divisor comun $\operatorname{GCD}(e,(p–1) \times (q–1))==1$
* Calcular el inverso modular de $e$, el cual llamaremos $d$. 

In [44]:
import math 

def isPrime(n):
    if n<2:
        return False
    for i in range(2,int(math.sqrt(n)+1)):
        if n % i == 0:
            return False
    return True

isPrime(9)
isPrime(11)

True

In [45]:
getPrimes = lambda n : [n for n in range(n) if isPrime(n)]

getPrimes(20)

[2, 3, 5, 7, 11, 13, 17, 19]

In [118]:
import random 

def generateLargePrime(keysize=1024):
    num=0
    while not isPrime(num):
        num = random.randrange(2**(keysize-1), 2**(keysize))
    return num

In [119]:
def generate_composite(keysize):
    # Creates public/private keys keySize bits in size.
    p = 0
    q = 0
    print('Generating p,q primes...')
    while p == q:
        p = generateLargePrime(keysize)
        q = generateLargePrime(keysize)
        n = p * q
    print('done!')
    return n,p,q

In [130]:
def gcd(x, y): 
   while(y): 
       x, y = y, x % y 
   return x 

def findModInverse(a, m):
    if gcd(a, m) != 1:
        return None # No mod inverse if a & m aren't relatively prime.
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # Note that // is the integer division operator.
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3),v1, v2, v3
    return u1 % m

In [138]:
def generate_rsa(keysize):
    n,p,q=generate_composite(keysize)
    while True:
        e = random.randrange(2**(keysize-1), 2**(keysize))
        if gcd(e,(p-1)*(q-1))==1:
            break
    print('p:',p)
    print('q:',q)
    print('e:',e)
    d = findModInverse(e, (p-1)*(q-1))
    print('d:',d)
    return (e,n),(d,n)

In [139]:
public_key,private_key=generate_rsa(20)

Generating p,q primes...
done!
p: 987599
q: 1020709
e: 709715
d: 517395127091


In [140]:
public_key

(709715, 1008051187691)

In [141]:
private_key

(517395127091, 1008051187691)

In [190]:
SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'

len(SYMBOLS)**169<2**1024

True